Acquired full statistical datasets distributed by the Dept. of Justice, can probably tease something out of the dates in connection with the different classes of cases.

**Questions**:
- What kinds of cases have ebbed and flowed over time?
- Are certain districts responsible for more of one type of crime than the other?
    - Can we drill down into the reasons that might surround those disparities?
    - How granular can we get? e.g. How big are the districts and what can we learn from the geographic separation?
- What kinds of cases is the Justice Dept. prosecuting? And who is getting sued in the government?

In [44]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import datetime
import os

Someone at the Dept. of Justice decided that the best way to format their data wasn't to use a delimited format, but rather to use the **actual character positions on each line** to identify unique fields. This is truly hellish and nonsense of the highest order, but that's why you'll see lots of subscripting ranges in the following code. I'll try to identify segments where I'm extracting particular values.

In [45]:
with open('justicedept_disks/DISK01/gs_case.txt' ,'r') as f:
    content = f.readlines()
len(content)

5770076

In [46]:
import calendar
month_abbrev_to_num = dict((v.lower(),k) for k,v in enumerate(calendar.month_abbr))
month_abbrev_to_num

{'': 0,
 'apr': 4,
 'aug': 8,
 'dec': 12,
 'feb': 2,
 'jan': 1,
 'jul': 7,
 'jun': 6,
 'mar': 3,
 'may': 5,
 'nov': 11,
 'oct': 10,
 'sep': 9}

In [56]:
#Extract values from case file
data = []

for case in content:
    try:
        #Date parsing
        datestring = case[64:75].strip()
        if len(datestring) < 11:
            case_received_date.append('')
            continue
        
        dateyear = int(datestring[-4:])
        datemonth = month_abbrev_to_num[datestring[3:6].lower()]
        dateday = int(datestring[0:2])
        
        full_date = datetime.date(
            dateyear,
            datemonth,
            dateday
        )
        
        #Try appending data
        data.append([
            case[13], #case_class
            case[0:3], #case_district
            full_date, #received_date
            case[3:13], #case_id
            case[81:83], #case_us_role
            case[77:81], #case_type
            case[157:182], #case_lead_charge
            case[245], #case_status
            case[271:282], #case_tot_victims
            case[86:88] #case_weight
        ])
    except IndexError:
        print(case)
        
len(data)

FLN1997V00629V*                                                 20-AUG-1997      D US   4                     97

FLN1997V00679V*                                                 04-SEP-1997      D US   1                     97



5770070

List of potential US Roles:
    - A = Amicus
    - AE = Appeal filed against
    - AN = U.S. has filed the Appeal
    - B = Creditor
    - C = Counsel cost Plus Contractor
    - D = U.S. is Defendant
    - G = Counsel for Government Employee
    - H = Third Party Defendant
    - I = Counsel for Native American
    - IN = Investigative Agency
    - L = Third Party Plaintiff
    - M = ? but apparently there are 802 of these cases.
    - O = Other
    - P = Plaintiff
    - T = Intervenor
    - V = Counsel for Veteran

In [51]:
#Dump all the data into a dataframe
cases = pd.DataFrame({
    'ID' : case_id,
    'Received_Date' : case_received_dateceived_date,
    'Class' : case_class,
    'Type' : case_type,
    'District' : case_district,
    'US_Role' : case_us_role,
    'Lead_Charge' : case_lead_charge,
    'Status' : case_status,
    'Tot_Victims': case_tot_victims
})

ValueError: arrays must all be same length